In [24]:
import cv2
import numpy as np
import pandas as pd
import imutils
import pickle
import cv2
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
bg = None

In [25]:
print(pickle.format_version)

4.0


In [26]:
def run_avg(image , aWeight):
    global bg
    if bg is None:
        bg = image.copy().astype('float')
        return 
    cv2.accumulateWeighted(image, bg , aWeight)

In [47]:
def segment(image, threshold = 40):
    diff = cv2.absdiff(bg.astype('uint8') , image)
    cv2.imshow("diff", diff)
    thresholded = cv2.threshold(image.copy(),threshold, 255 , cv2.THRESH_BINARY)[1]
    thresholded = cv2.threshold(diff,threshold, 255 , cv2.THRESH_BINARY)[1]
    cv2.imshow("thresholded", thresholded)
    (cnts, _) = cv2.findContours(thresholded.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    #(cnts, _) = cv2.findContours(diff.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key = cv2.contourArea)
        return (diff , segmented)

In [28]:
ls

 Volume in drive D is Data
 Volume Serial Number is 305E-8A43

 Directory of D:\git\hand_gesture_recog\hand_gesture

19-11-2020  12:30    <DIR>          .
19-11-2020  12:30    <DIR>          ..
05-11-2020  13:11    <DIR>          .ipynb_checkpoints
05-11-2020  11:58         6,369,966 _model_.sav
06-11-2020  05:24    <DIR>          HAND_DETECTION
19-11-2020  12:30             8,121 hand_detection.ipynb
06-11-2020  05:26            44,663 hand_recog_model.ipynb
               3 File(s)      6,422,750 bytes
               4 Dir(s)  794,513,149,952 bytes free


In [29]:
loc = r'D:\datasets\archive\leapGestRecog\0'

In [30]:
lookup = {'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [31]:
lookup

{'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [32]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir(loc + str(0)):
    if not j.startswith('.'):                              
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [48]:
if __name__ == "__main__":    
    aWeight = 0.2    
    camera = cv2.VideoCapture(0)    
    model = tf.keras.models.load_model('HAND_DETECTION')
    top, right, bottom, left = 10, 350, 225, 590    
    num_frames = 0  
    predict_img = None
    result = None
    while(True):       
        (grabbed, frame) = camera.read()        
        frame = cv2.flip(frame, 1)        
        
        clone = frame.copy()       
        (height, width) = frame.shape[:2]        
        roi = frame[top:bottom, right:left]    
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        #gray = cv2.GaussianBlur(gray, (7,7), 0)              
        cv2.imshow("before avg", gray)
        if num_frames < 50:
            run_avg(gray, aWeight)
        else:           
            hand = segment(gray)            
            if hand is not None:                
                (thresholded, segmented) = hand                
                cv2.drawContours(clone, [segmented + (right, top)], -1, (255, 142, 77) , 3)
                predict_img = thresholded      
                #predict_img = cv2.resize(predict_img , (120,320))
                predict_img = cv2.resize(predict_img , (320,120))
                cv2.imshow("image prediction img", predict_img)
                predict_img = np.array(predict_img, dtype = 'float32')
                y_pred = model.predict(predict_img.reshape(1,120,320,1))
                result = np.argmax(y_pred)
                
                result = reverselookup[result]               
                 
                
        font = cv2.FONT_HERSHEY_SIMPLEX        
        #color = (255, 255, 255)
        color = (0, 0, 0)
        i = 0
        cv2.putText(clone,result,(350, 250),font, 1,color,2,cv2.LINE_AA)
            
            
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)        
        num_frames += 1   
         
        
        #print(result)
        cv2.imshow("Video Feed", clone)        
        keypress = cv2.waitKey(1)       
        if keypress == ord("q"):
            break
camera.release()
cv2.destroyAllWindows()